**PREAMBLE:** This notebook isn't the most useful but it might prove informative.  Basically, what happened was that I realized that the data was a lot messier then I expected.  I receieved the latest data during a time in which the Q&A section was going through a change in the way it was stored in the database.  This led to a situation in which the data had some doubles and it took me a while to sort out why.  THe reason is explained in [this question](https://publiclab.org/questions/bsugar/04-15-2019/how-are-the-q-a-s-tied-together-in-the-website-database) on the Public Lab website.  The other thing that happened is that I realized how many of the users were spam users and began to explore ways of removing them.  So, I moved the entire process of dealing with spam over to a different notebook, USER SPAM REMOVAL.

**NOTE:** The following contains a few things that I do out of habit when working with data, but they are unneccessary in this case.  

I will explain them since they might be useful for the reader in a different project but _**I HAD NO PROBLEM RUNNING ANY OF THESE NOTEBOOKS WITH THE FULL FILES ON A COMPUTER WITH 2GB OF RAM**_  _At anytime, feel free to skip to the first section, "MERGE OLD AND NEW DATA"._   

If you're really interested in how to take a very large file and get a random sample, you can find all of the bash commands in the following notebook in the  section entitled [PREPARING A FILE SAMPLE IN BASH](https://render.githubusercontent.com/view/ipynb?commit=e3b6ada097105708a6ed32f146e1e4657c387393&enc_url=68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f736b696c66756c6c796375726c65642f64706c612f653362366164613039373130353730386136656433326631343665316534363537633338373339332f42756c6b253230446f776e6c6f61642e6970796e62&nwo=skilfullycurled%2Fdpla&path=Bulk+Download.ipynb&repository_id=138903471&repository_type=Repository#PREPARING-A-FILE-SAMPLE-IN-BASH).  Since I did a much simpler process here, I'll show the few commands I used along the way.

**1. OBTAINING THE SIZE OF THE FILES**

Here are some stats about the files.  I wanted to know these because sometimes when I work with very large files, I will take samples, make sure my code is solid, and then run them with the larger files on a more powerful computer.  

You can of course take a sample of the file once it is loaded, but if the file is very large, then you want to do so before hand.  

To find out how large a file is in terms of filesize, go into the folder that the files are in and type:  

**[~/path/to/files] user ->** du -sh &nbsp; *

In [ ]:
792K	answers.csv
3.8M	comments.csv
488K	notes.csv
204K	questions.csv
50M	users.csv
44K	wikis.csv

**2. OBTAINING THE NUMBER OF LINES IN A FILE**


**Note:** _If you are using JSON data which does not have one line per json object, then this method won't be as useful.  Additionally, because Public Lab's data is stored in markdown converted to HTML, it contains the escape sequences for carraige returns and new lines (\r and \n) and will also throw off your count of lines._

To find out the number of lines in each file, use the following bash command:

**[~/path/to/files] user ->** wc -l &nbsp;   *

In [ ]:
7399 answers.csv
36752 comments.csv
2105 notes.csv
821 questions.csv
382952 users.csv
305 wikis.csv
430334 total

**3. CREATE SOME SAMPLE FILES OF VARYING SIZES**

What I like to do is then take the top and bottom of a certain number of lines in each file and label them as powers of 10.  For example, if I want a file with 1000 lines I'll use the following bash commands:

**[~/path/to/files] user ->** head -n -500 full_file.csv >> sample_file_1e3.csv  
**[~/path/to/files] user ->** tail -n -500 full_file.csv >> sample_file_1e3.csv # USE DOUBLE ARROW FOR APPEND!  
**[~/path/to/files] user ->** wc -l sample_file_1e3.csv # SHOULD RETURN 1000 LINES BELOW  
1000 same_file_1e3.csv 

In [1]:
# PACKAGES WE'LL NEED
import pandas as pd
import numpy as np
import re
import csv

print 'IMPORTED!'

IMPORTED!


Here's where all of the samples and file sizes come into play.  I create a bunch of variables so that I don't have to rewrite the filename in each situation.  _Again, this whole thing is unnecessary for this project._

1) I have named by data type e.g., users, wikis, notes, etc.).  

2) The second variable is a boolean which I set depending upon which computer I'm using.  Again, you don't need to do this for this project, but if you have a setup where you're going to run samples on a smaller machine, then this will switch the statement below so that it either uses the samples or the full files.  

3) _m_ and _exp_ are the power of 10 setup.  In the case below, that will end up being _25e2_ when the file name is created.  

Then, I can create a file name which is: **data\_type + '\_' + m + 'e' + exp + '.csv'** and if I want a new file, all I need to do is change one of those variables.  See comment below.

In [3]:
sample = False

data_type = 'users'

m = 25
exp = 2

if sample:
    csv_dir = '../data/2016_2019/samples/'
    e = '_' + str(m) + 'e' + str(exp) # if m = 25 and exp = 2, will become '_25e2
    csv_path = csv_dir + data_type + e + '.csv' # '../data/2016_2019/samples/' + 'users' + '_25e2' + 'csv'
else:
    csv_dir = '../data/2016_2019/full/'
    e = ''
    csv_path = csv_dir + data_type + e + '.csv' # '../data/2016_2019/samples/' + 'users' + '' + '.csv'

In order to prevent the need to do this for every type of file, I have created a list of all the different types of files, and a function (see section 5.1 "CREATE CSV PATHS") which takes this list, and does exactly what the above does but for multiple file types instead of just one.

In [ ]:
data_types = ['answers', 'comments', 'notes', 'questions', 'users', 'wikis', 'paths', 'tags', 'terms']

csv_paths = create_csv_paths(data_types)

# print csv_paths

From here on out, I will generally write what's happening inside the code as comments beginning with '#'.  If there is something a bit deeper going on, I'll explain it more fully.

## MERGE OLD AND NEW DATA

### NODES: 03-2013 - 08-2016

#### SEPPERATE QUESTIONS FROM NOTES AND CONCAT NOTES PRE/POST TOGETHER

In [29]:
use_cols = [u'nid', u'type', u'title', u'uid', u'created', u'comment', u'tnid', u'cached_likes']
notes_path_one = '../data/2016_2019/full/notes_part_one.csv'
notes_one = pd.read_csv(notes_path_one, usecols=use_cols) 
notes_one.sort_values('created', inplace=True)

# Tuesday, December 7, 2010 7:19:19 
notes_one.head()

,nid,type,title,uid,created,comment,tnid,cached_likes
0,6,note,Toy spectrometers arrived,1,1291706359,2,0,0
1,7,note,Attempted infrared imaging of motor oil in sal...,1,1291733403,2,0,0
2,13,note,Bayou St. Denis in Cartagen Knitter,1,1291830863,2,0,0
3,17,note,Scanning spectrometry video test,1,1291944246,2,0,0
4,23,note,Normalized Difference Vegetation Index,1,1292030311,2,0,0


In [57]:
notes_one['type'].unique()

array(['note'], dtype=object)

In [31]:
# Wednesday, July 27, 2016 11:37:33 
notes_one.tail()

,nid,type,title,uid,created,comment,tnid,cached_likes
2628,13309,note,Progress of the Expanded Q & A Project,468506,1469477988,2,0,3
2629,13310,note,Photos to 3D,554,1469504400,2,0,0
2625,13311,note,Last week in Google Summer of Code,1,1469554219,2,0,0
2627,13312,note,Question: Joining,483438,1469617059,2,0,0
2626,13313,note,Top Dating Agency in Pune !,485226,1469619453,2,0,0


In [61]:
# notes_one.loc[notes_one['nid'] == 11503]
notes_two.loc[notes_two['nid'] == 13313]
notes_two['type'].unique()

array(['note'], dtype=object)

In [32]:
notes_path_two = '../data/2016_2019/full/notes_part_two.csv'
notes_two = pd.read_csv(notes_path_two, usecols=use_cols)#, usecols=use_cols, parse_dates=['created_at'], na_filter=False) 
notes_two.sort_values('created', inplace=True)

# Saturday, January 3, 2015 23:36:32 
notes_two.head()

,nid,type,title,uid,created,comment,tnid,cached_likes
0,11503,note,Public Lab at Science for Action,554,1420328192,2,0,3
1,11504,note,"Public Lab Barnraising 2014, day 3",554,1420328228,2,0,0
2,11505,note,"Public Lab Barnraising 2014, day 2",554,1420328254,2,0,0
3,11506,note,"Public Lab Barnraising 2014, day 1",554,1420328276,2,0,0
4,11507,note,"Public Lab Organizers Summit, 2014",554,1420328303,2,0,4


In [33]:
# Sunday, April 14, 2019 10:27:03 
notes_two.sort_values('created').tail()

,nid,type,title,uid,created,comment,tnid,cached_likes
3150,19046,note,"Does the ""rolling shutter"" aspect of cameras i...",505738,1555095200,2,0,1
3151,19047,note,How to interpret soil test results,569208,1555103090,2,0,0
3152,19050,note,Prototyping A Cellphone Microscope,237313,1555165221,2,0,0
3153,19051,note,GSOC-19 Mapknitter synchronous editing,517093,1555184525,2,0,0
3154,19054,note,Forest Conservation Through Local Wisdom,584138,1555237623,2,0,0


In [14]:
for data_type in ['page', 'note', 'event', 'tool', 'map']:
    
    df = nodes.loc[nodes['type'] == data_type]
    print data_type + ':', df.shape[0]
    df.to_csv('../data/2016_2019/full/' + data_type + 's_part_one', index=False)

page: 908
note: 2635
event: 4
tool: 13
map: 230


In [41]:
quest_cols = [u'nid', u'uid', u'qid', u'created', u'comment', u'comments_count', u'cached_likes', u'type', u'title']

renamed_quest_cols = {'created': 'date', 'comments_count': 'thread_cmnts', 
                      'comment': 'user_cmnts', 'cached_likes': 'likes'}

questions = pd.read_csv(csv_paths['questions'])
questions['type'] = 'qstn'
questions['qid'] = range(1, questions.shape[0] + 1)

questions = questions[quest_cols] # questions.columns = cols
questions.rename(index=str, columns=renamed_quest_cols, inplace=True)

# questions.columns
questions.head() # (902, 8)

,nid,uid,qid,date,user_cmnts,thread_cmnts,likes,type,title
0,11516,433692,1,1421053078,2,3,1,qstn,Is it possible to log the data from a dustduin...
1,11524,433790,2,1421278243,2,1,1,qstn,Is the dustduino (with Shinyei sensor) appropr...
2,11526,433849,3,1421429978,2,2,0,qstn,How can I test my tap water for BPA
3,11545,4,4,1422557787,2,6,5,qstn,dealing with patents: Black & Decker and the T...
4,11610,435436,5,1424457388,2,2,0,qstn,Is the project still active?


In [44]:
notes_only_one = notes_one.loc[~notes_one['nid'].isin(questions['nid'])]
notes_only_two = notes_two.loc[~notes_two['nid'].isin(questions['nid'])]

print notes_one.shape[0], notes_two.shape[0], questions.shape[0]
print notes_only_one.shape[0], notes_only_two.shape[0]

2635 3155 902
2573 2253


In [50]:
notes_only_one.sample(10)

,nid,type,title,uid,created,comment,tnid,cached_likes
1322,7436,note,Picking Trouble-Free Systems In bags,63504,1368287941,2,0,2
1900,12523,note,Oil Testing Kit -- Beta tests (with some neede...,430549,1450304007,2,0,0
597,2824,note,FABAnorth arrow,1310,1342462136,2,0,0
114,349,note,QGIS 1.7 is out :),12,1308595040,2,0,2
2085,12093,note,Tampon Testing for Wastewater,429279,1437665088,2,0,3
1847,12460,note,GroundWorksNOLA youth presenting at Barnraisin...,7,1448739424,2,0,0
973,6235,note,Green Slice: shots from kite lesson in Mr Mor...,379,1362698675,2,0,0
1853,12464,note,Riffle Beta Release: Feature Requests / Review?,43651,1448926026,2,0,3
780,4760,note,Barnraising: Stewart talks map making,4,1351956635,2,0,0
1402,7586,note,moncler outlet qcgykt,67835,1368645020,2,0,2


In [53]:
notes_only_two.sample(10)

,nid,type,title,uid,created,comment,tnid,cached_likes
1161,13457,note,Draft Meta-Riffle Part 2: All about pins,459085,1474064170,2,0,1
950,13129,note,Survey for newcomers on interface design of ne...,1,1463677254,2,0,1
2834,17387,note,Educating a New Generation of Advocates,540844,1540408226,2,0,3
967,13157,note,My cat helped me out on this one!,461120,1464797328,2,0,0
1189,13501,note,MapKnitter map of Joshua Leslie Lab 5,493191,1475102830,2,0,0
1847,14847,note,Wrapping up Google Summer of Code 2017 at Publ...,1,1504821106,2,0,3
1676,14501,note,Calling for your favorite Kite/Balloon Mapping...,237313,1496681396,2,0,4
2869,17476,note,Expanding Public Lab's community stats system,558157,1541478965,2,0,3
2936,17879,note,Convert SD1000 to infra-red,460991,1544639573,2,0,0
699,12642,note,a pendulum rig flies over China,7,1454362851,2,0,4


In [63]:
notes_only = pd.concat([notes_only_one, notes_only_two])
notes_only.to_csv(csv_paths['notes'], index=False)
notes_only.sample(10)

,nid,type,title,uid,created,comment,tnid,cached_likes
600,12498,note,Oil Test Kit: 3 crude oil scans,7,1449865524,2,0,0
1008,13226,note,Plab v2.5 (upgrade 2) Comparison Spectra to Sp...,461120,1466604762,2,0,0
2009,12622,note,Decolonized Skies,152,1453837584,2,0,6
202,11834,note,Mid stream Urine analysis: Check whether you n...,442298,1432393019,2,0,2
2858,17448,note,Inexpensive conductivity probe Step 1 - Sens...,556121,1541004921,2,0,3
372,12177,note,HackeAr,426547,1440723379,2,0,1
93,11649,note,PLab 3 Spectrometer Upgrade Prototype,54025,1425288210,2,0,6
2896,17715,note,Our usb microscop design,546059,1542686231,2,0,2
2942,17891,note,Revised 3D printable mini spectrometer,563397,1544745656,2,0,3
179,11800,note,Community formaldehyde monitoring at natural g...,380298,1431112908,2,0,2


#### SEPPERATE ANSWERS FROM COMMENTS AND CONCAT COMMENTS PRE/POST TOGETHER

In [66]:
cmnt_cols = ['nid', 'uid', 'cid', 'aid', 'timestamp', 'type', 'thread', 'comment', 'reply_to']
comments = pd.read_csv(csv_paths['comments'])
comments['type'] = 'cmnt'
comments['reply_to'].fillna(0, inplace=True)
comments['reply_to'] = comments['reply_to'].astype('int64')
comments = comments[cmnt_cols]
comments.rename(index=str, columns={'timestamp': 'date'}, inplace=True)

In [67]:
comments.sort_values('date', inplace=True)
comments['date'][0], comments['date'][-1] # January 2, 2015 Friday - April 13, 2019

(1420182880, 1555185829)

In [ ]:
comments.tail()

In [65]:
ans_cols = [u'nid', u'uid', u'id', u'created_at', u'cached_likes', u'type', u'content', u'accepted']

answers = pd.read_csv(csv_paths['answers'])
answers['type'] = 'ans'
answers = answers[ans_cols]

renamed_ans_cols = {'id': 'aid', 'created': 'date', 'cached_likes': 'likes'}
answers.rename(index=str, columns=renamed_ans_cols, inplace=True)

# answers.columns
answers.sample(5)  #(1081, 8)

,nid,uid,aid,created_at,likes,type,content,accepted
206,14061,504525,218,2017-03-26 14:50:20 UTC,0,ans,The Photo Monitoring plugin already has a comm...,True
620,15279,1,726,2018-01-22 20:13:55 UTC,0,ans,[![61UCStnHAzL._SL1001_.jpg](https://publiclab...,False
209,14068,423961,221,2017-03-30 06:38:09 UTC,1,ans,"@alanrday I don't know if this is helpful, but...",False
654,15594,54025,762,2018-01-29 22:49:23 UTC,0,ans,Questions are good ;-)\r\n\r\nReading between ...,True
1068,18170,237313,1335,2019-01-24 21:36:33 UTC,0,ans,There is an overview of some of they symptoms...,True


In [ ]:
no_comments_one = notes_only_one.loc[~notes_only_one['nid'].isin(comments['nid'])]
no_comments_two = notes_only_two.loc[~notes_only_two['nid'].isin(comments['nid'])]

print notes_one.shape[0], notes_two.shape[0], comments.shape[0]
print notes_only_one.shape[0], notes_only_two.shape[0]
print no_comments_one.shape[0], no_comments_two.shape[0]

### USERS: 03-2013 - 08-2016

In [219]:
rusers_path = '../data/2016_2019/full/users_03_2014-08_2016.csv'
use_cols = ['id', 'username', 'created_at']
rusers = pd.read_csv(rusers_path, usecols=use_cols, parse_dates=['created_at'], na_filter=False) 
rusers.sort_values('created_at', inplace=True)
rusers.head()

,id,username,created_at
0,1,warren,2013-03-14 16:16:26
2,4,mathew,2013-04-08 01:46:32
6,12,gonzoearth,2013-04-08 02:10:49
3,7,liz,2013-04-11 15:26:02
5,9,Shannon,2013-04-11 15:26:24


In [223]:
rusers.reset_index(inplace=True, drop=True)
rusers.head()

,id,username,created_at
0,1,warren,2013-03-14 16:16:26
1,4,mathew,2013-04-08 01:46:32
2,12,gonzoearth,2013-04-08 02:10:49
3,7,liz,2013-04-11 15:26:02
4,9,Shannon,2013-04-11 15:26:24


In [ ]:
use_cols = ['id', 'username', 'created_at', 'bio']
users = pd.read_csv(csv_paths['users'], usecols=use_cols, parse_dates=['created_at'], na_filter=False) # na_filter removes NaN's when empty string
users.rename(columns={'id': 'uid'}, inplace=True)

print 'ROWS:', users.shape[0]
print 'COLUMNS:', users.shape[1]

## USER SPAM REMOVAL

In [8]:
use_cols = ['id', 'username', 'created_at', 'bio']
users = pd.read_csv(csv_paths['users'], usecols=use_cols, parse_dates=['created_at'], na_filter=False) # na_filter removes NaN's when empty string
users.rename(columns={'id': 'uid'}, inplace=True)

print 'ROWS:', users.shape[0]
print 'COLUMNS:', users.shape[1]

ROWS: 94059
COLUMNS: 4


### REMOVE \r AND \n

In [ ]:
# GET INDEX NO. OF TEN BIO'S THAT CONTAIN STRING TO BE REPLACED
users[users['bio'].str.contains('\r|\n') == True].head(10).index # 

In [ ]:
# REPLACE STRINGS
users['bio'] = users['bio'].str.replace('\r', '|R').str.replace('\n', '|N')

In [ ]:
# RETEST TO SEE IF IT WORKED
test_list = [0, 9, 11, 25, 47, 61, 62, 65, 67, 70]
users.loc[users.index[test_list], 'bio'].str.contains('\r|\n') # SHOULD BE ALL FALSE

### ADD COLUMN: BIO_LENGTH

In [47]:
users['bio_length'] = users['bio'].str.len()
users.head()

,id,username,created_at,bio,bio_length
0,372,John_Wells,2016-09-09 14:17:26+00:00,"Kite aerial photographer with, and co-founder ...",153
1,46537,deeryezi,2016-08-02 03:23:22+00:00,,0
2,481536,Leonwgy,2016-07-01 00:02:28+00:00,"<a href=""http://dekoracjadomu.pl/koopman-wazon...",117
3,481537,Mickmmt,2016-07-01 00:07:39+00:00,"<a href=""http://deasalus.eu/stomatologiczna"">h...",83
4,481538,Marielwosp,2016-07-01 00:10:17+00:00,"<a href=""http://sk.sizexxlmatter.com"">http://s...",293


### ADD COLUMN: HAS_URL

In [ ]:
url_pattern = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\), ]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
re_url = re.compile(url_pattern)

In [33]:
has_url_str = 'this should return true https://dekoracjadomu.pl/koopman-wazon-geometryczny-ceramika-golebi'
no_url_str = 'this should return false dekoracjadomu.pl/koopman-wazon-geometryczny-ceramika-golebi'
lrg_url_str= '<a href=""http://sk.sizexxlmatter.com"">http://sk.sizexxlmatter.com</a> A special mention to put natural cures, substances that would generally be located in your body along with which is often used while dietary supplements to aid get the necessary ranges to take place.'

# [bool(re_url.search(url)) for url in [has_url_str, no_url_str, lrg_url_str]]
# [True, False, True]

In [48]:
users['has_url'] = users['bio'].str.contains(re_url, na=False)
users.head()

,id,username,created_at,bio,bio_length,has_url
0,372,John_Wells,2016-09-09 14:17:26+00:00,"Kite aerial photographer with, and co-founder ...",153,True
1,46537,deeryezi,2016-08-02 03:23:22+00:00,,0,False
2,481536,Leonwgy,2016-07-01 00:02:28+00:00,"<a href=""http://dekoracjadomu.pl/koopman-wazon...",117,True
3,481537,Mickmmt,2016-07-01 00:07:39+00:00,"<a href=""http://deasalus.eu/stomatologiczna"">h...",83,True
4,481538,Marielwosp,2016-07-01 00:10:17+00:00,"<a href=""http://sk.sizexxlmatter.com"">http://s...",293,True


### ADD COLUMN: BEGINS WITH URL

In [49]:
a_href = users['bio'].str.startswith('<a href=') 
http = users['bio'].str.startswith('http')
users['begins_url'] = (a_href | http)

users.head()

,id,username,created_at,bio,bio_length,has_url,begins_url
0,372,John_Wells,2016-09-09 14:17:26+00:00,"Kite aerial photographer with, and co-founder ...",153,True,False
1,46537,deeryezi,2016-08-02 03:23:22+00:00,,0,False,False
2,481536,Leonwgy,2016-07-01 00:02:28+00:00,"<a href=""http://dekoracjadomu.pl/koopman-wazon...",117,True,True
3,481537,Mickmmt,2016-07-01 00:07:39+00:00,"<a href=""http://deasalus.eu/stomatologiczna"">h...",83,True,True
4,481538,Marielwosp,2016-07-01 00:10:17+00:00,"<a href=""http://sk.sizexxlmatter.com"">http://s...",293,True,True


### DISTRIBUTIONS TO CHOOSE CUT OFFS

In [44]:
# SELECT ALL USERS WITH A URL IN THEIR BIO
url = users['has_url'] == True
bio_length = users['bio_length'] > 0
has_url = users.loc[url & bio_length]
num_has_url = has_url.shape[0]

# OF THOSE WITH A URL IN THEIR BIO, WHICH BIO'S BEGIN WITH ONE
a_href = has_url['bio'].str.startswith('<a href=') 
http = has_url['bio'].str.startswith('http')
has_url['begins'] = (a_href | http)
num_begins_url = has_url.loc[has_url['begins'] == True].shape[0]

url = users['has_url'] == False
bio_length = users['bio_length'] > 0
no_url = users[url & bio_length]
no_url.shape

print 'BIO HAS URL:', num_has_url
print 'BIO BEGINS WITH URL:', num_begins_url
print 'BIO DOES NOT BEGIN WITH URL:', 
print 'BIO LENGTH MEAN, MED, STD'

USERS WITH A URL IN THE BIO: 73854


In [192]:
q = pd.read_csv(csv_paths['questions'], usecols=['nid', 'uid', 'type', 'legacy_views', 'views', 'title'])
n = pd.read_csv(csv_paths['notes'], usecols=['nid', 'uid', 'type', 'legacy_views', 'views', 'title'])

In [193]:
q.loc[q['nid'] == 13745]

,nid,type,title,uid,legacy_views,views
80,13745,note,Is anyone doing any work with fungi? or biorem...,498969,207,408


In [194]:
n.loc[n['nid'] == 13745]

,nid,type,title,uid,legacy_views,views
349,13745,note,Is anyone doing any work with fungi? or biorem...,498969,207,408


In [67]:
# users[(users['bio_length'] > 4) & (users['bio_length'] <= 5)].sort_values('bio_length').head(50)

In [84]:
quests = pd.read_csv(csv_paths['questions'], usecols=['nid', 'uid', 'type', 'title'])
quests.head()
# notes_cols = ['nid','type','title','uid']
# notes = pd.read_csv(csv_paths['notes'], usecols=notes_cols)
# notes = notes[['nid', 'uid', 'type', 'title']]
# notes.head()

,nid,type,title,uid
0,13252,note,Desktop spectrometry 3.0 kit: Can it quantify/...,481667
1,13256,note,Question: Does the Levitation Light dive for y...,4
2,13286,note,Spectrometer from a drone,483452
3,13312,note,Question: Joining,483438
4,13358,note,How to neutralize/remove the source's spectra ...,460486


In [81]:
notes['type'].unique()

array(['note'], dtype=object)

In [85]:
notes.loc[notes['nid'] == 13745]

,nid,uid,type,title
349,13745,498969,note,Is anyone doing any work with fungi? or biorem...


In [101]:
wikis = pd.read_csv(csv_paths['wikis'], usecols=['nid', 'uid', 'type', 'title', 'comments_count', 'vid', 'slug'])
wikis.loc[wikis['comments_count'] == 2].head()

,nid,vid,type,title,uid,comments_count,slug
29,13654,37175,page,Translation,7,2,translation
100,14798,34082,page,Open Call,1,2,open-call
103,14838,30652,page,Irma,7,2,irma
131,15029,35607,page,sandbox-2,1,2,sandbox-2
263,17494,35430,page,Comment templates,1,2,comment-templates


In [87]:
wikis['type'].unique()

array(['page'], dtype=object)

In [88]:
comments = pd.read_csv(csv_paths['comments'])#, usecols=cols)
comments.head()

,cid,pid,nid,uid,subject,comment,hostname,timestamp,status,format,thread,name,mail,homepage,aid,comment_via,message_id,tweet_id,reply_to
0,14837,0,12876,447491,NaN,Just a catchup here:\r\n\r\nI've continued to ...,NaN,1467331379,1,1,02/,NaN,NaN,NaN,0,0,NaN,NaN,NaN
1,14838,0,13248,461120,NaN,"@stoft hey Dave, I thought we never say ""oops,...",NaN,1467363313,1,1,02/,NaN,NaN,NaN,0,0,NaN,NaN,NaN
2,14839,0,13248,461120,NaN,@stoft Hey Dave I thought you might find this ...,NaN,1467382535,1,1,02/,NaN,NaN,NaN,0,0,NaN,NaN,NaN
3,14840,0,13248,461120,NaN,@stoft hey dave I'm including the 2 plots agai...,NaN,1467382846,1,1,02/,NaN,NaN,NaN,0,0,NaN,NaN,NaN
4,14841,0,13248,54025,NaN,Unfortunately ..... while photomultipliers res...,NaN,1467396699,1,1,02/,NaN,NaN,NaN,0,0,NaN,NaN,NaN


In [100]:
comments.loc[comments['nid'] == 14798]

,cid,pid,nid,uid,subject,comment,hostname,timestamp,status,format,thread,name,mail,homepage,aid,comment_via,message_id,tweet_id,reply_to
4983,20385,0,14798,540243,NaN,Hello! I just found this page on the weekly op...,NaN,1534198414,1,1,01/,NaN,NaN,NaN,0,0,NaN,NaN,NaN
4984,20386,0,14798,7,NaN,yes it is! Hope to see you on tomorrow's call!...,NaN,1534203301,1,1,02/,NaN,NaN,NaN,0,0,NaN,NaN,NaN


In [102]:
comments.loc[comments['nid'] == 13312]

,cid,pid,nid,uid,subject,comment,hostname,timestamp,status,format,thread,name,mail,homepage,aid,comment_via,message_id,tweet_id,reply_to
191,15037,0,13312,7,NaN,"HI Stephen, \r\nThere are a couple folks in Au...",NaN,1469627201,1,1,01/,NaN,NaN,NaN,0,0,NaN,NaN,NaN
414,15290,0,13312,483438,NaN,"Hi Liz, will do, also happy to remove my email.",NaN,1472784170,1,1,02/,NaN,NaN,NaN,0,0,NaN,NaN,NaN
6389,22245,0,13312,468506,NaN,Sorry to bother you. I was just testing the ne...,NaN,1470129369,1,1,/01,NaN,NaN,NaN,0,0,NaN,NaN,NaN


## PATHS

I put together this list of paths and the accompanying function at **4.1 PATH TO URL** because I wanted an easy way to make a URL from the data when I need to see an actual page.  So the function takes a list of paths

In [202]:
path_cols = ['type', 'nid', 'created', 'path']

wiki_paths = pd.read_csv(csv_paths['wikis'])#, usecols=['created', 'nid', 'path'])
wiki_paths['type'] = 'wiki'
wiki_paths = wiki_paths[path_cols]

notes_paths = pd.read_csv(csv_paths['notes'])#, usecols=['created', 'nid', 'path'])
notes_paths['type'] = 'note'
notes_paths = notes_paths[path_cols]

quest_paths = pd.read_csv(csv_paths['questions'])#, usecols=['created', 'nid', 'path'])
quest_paths['type'] = 'qstn'
quest_paths = quest_paths[path_cols]

all_paths = pd.concat([wiki_paths, notes_paths, quest_paths], ignore_index=True)
all_paths.rename(index=str, columns={'created': 'date'}, inplace=True)
all_paths = all_paths.to_csv(csv_paths['paths'], index=False)

In [201]:
# NOTE ALL PATHS REDIRECT IF NECCESSARY
paths = pd.read_csv(csv_paths['paths'])
paths.sample(5)

,type,nid,date,path
1465,note,13131,1463694241,/notes/dhaffnersr/05-19-2016/chemical-analysis...
734,note,11920,1433087744,/notes/peteryoung/05-31-2015/mapknitter-map-of...
3870,qstn,13962,1488131914,/notes/tim_spec/02-26-2017/jdepc-ov05-camera-i...
310,wiki,14806,1504192174,/wiki/harvey
3685,qstn,11946,1433350344,/notes/mattiecaroline1991/06-03-2015/question-...


## FUNCTIONS

### PATH TO URL

In [2]:
def path_to_url(nid, data_type, paths):
    
    try:

        path_type = (paths['type'] == data_type)
        path_nid = (paths['nid'] == nid)

        path = paths.loc[path_type & path_nid]['path'].values[0]

        return 'https://publiclab.org' + path
    
    except:
        
        return False
    
node_ids = [10388, 456789, 13252, 456789, 13252, 456789]
data_types = ['wiki', 'wiki', 'notes', 'notes','questions', 'questions']

for node_id, data_type in zip(node_ids, data_types):
    print path_to_url(node_id, data_type, paths)
# SHOULD BE URL, FALSE, URL, FALSE, URL, FALSE

NameError: name 'paths' is not defined

### CREATE CSV PATHS

In [2]:
def create_csv_paths(data_types, base='', exp='', full=True):
    
    csv_paths = {}
    
    for data_type in data_types:

        if not full:
            csv_dir = '../data/2016_2019/samples/'
            e = '_' + str(base) + 'e' + str(exp)
            csv_path = csv_dir + data_type + e + '.csv'
        else:
            csv_dir = '../data/2016_2019/full/'
            e = ''
            csv_path = csv_dir + data_type + e + '.csv'
            
        csv_paths[data_type] = csv_path
    
    return csv_paths

data_types = ['answers', 'comments', 'notes', 'questions', 'users', 'wikis']
csv_paths = create_csv_paths(data_types)

### SLUG TO URL (TURNED OUT TO BE UNNECESSARY)

In [181]:
def slug_to_url(nid, data_type, slugs):
    
    try:
    
        slug_type = (slugs['type'] == data_type)
        slug_nid = (slugs['nid'] == nid)

        db_slug = slugs.loc[slug_type & slug_nid]['slug'].values[0]

        if (data_type == 'questions') or (data_type == 'notes'):

            parts = db_slug.split('-', 1)

            user = parts[0]
            date = parts[1][:10]
            title = parts[1][11:]

            slug = '/'.join([user, date, title])

        elif data_type == 'wiki':

            slug = db_slug

        return 'https://publiclab.org/' + data_type + '/' + slug
    
    except:
        
        return False

# node_ids = [13252, 456789, 13252, 456789, 13252, 456789]
# data_types = ['wiki', 'wiki', 'notes', 'notes','questions', 'questions']

# for node_id, data_type in zip(node_ids, data_types):
    
#     print slug_to_url(node_id, data_type, test_slugs)

#### SLUG TESTS

In [196]:
slug_cols = ['nid', 'type', 'slug']

wiki_slugs = pd.read_csv(csv_paths['wikis'], usecols=['nid', 'slug'])
wiki_slugs['type'] = 'wiki'
wiki_slugs = wiki_slugs[slug_cols]

notes_slugs = pd.read_csv(csv_paths['notes'], usecols=['nid', 'slug'])
notes_slugs['type'] = 'notes'
notes_slugs = notes_slugs[slug_cols]

quest_slugs = pd.read_csv(csv_paths['questions'], usecols=['nid', 'slug'])
quest_slugs['type'] = 'questions'
quest_slugs = quest_slugs[slug_cols]

all_slugs = pd.concat([wiki_slugs, notes_slugs, quest_slugs], ignore_index=True)
all_slugs = all_slugs.to_csv('../data/2016_2019/full/slugs.csv', index=False)

In [187]:
# slugs = pd.read_csv('../data/2016_2019/full/slugs.csv')
slugs.sample(5)

,nid,type,slug
2992,16163,questions,NaN
224,16575,wiki,api
2524,13893,questions,fez-02-01-2017-diy-methods-for-nitrate-detecti...
298,18872,wiki,cafo
1090,14773,notes,NaN


In [ ]:
https://publiclab.org/stats?start=01-07-2013&end=01-07-2016&commit=Go
https://publiclab.org/stats?start=01-07-2012&end=01-07-2016&commit=Go